In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


/opt/homebrew/Caskroom/mambaforge/base/envs/hertel_pc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import pandas as pd
import json





In [8]:
def make_df_from_jsonl(file_path):
    # Initialize an empty list to store the parsed data
    data = []
    
    # Open the file and read line by line
    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                # Parse each line as a JSON object and append to the list
                json_data = json.loads(line)
                data.append(json_data)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON on line {line_number}: {e}")
                continue  # Skip this line or use 'break' if you want to stop processing altogether
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)

In [13]:
# Path to your JSONL file
file_path = 'training_samples.jsonl'

# Create DataFrame
df = make_df_from_jsonl(file_path)
print(df.head())

Error parsing JSON on line 35: Extra data: line 1 column 382 (char 381)
Error parsing JSON on line 41: Extra data: line 1 column 428 (char 427)
Error parsing JSON on line 42: Extra data: line 1 column 421 (char 420)
Error parsing JSON on line 46: Extra data: line 1 column 424 (char 423)
Error parsing JSON on line 58: Extra data: line 1 column 505 (char 504)
Error parsing JSON on line 64: Extra data: line 1 column 427 (char 426)
Error parsing JSON on line 67: Extra data: line 1 column 416 (char 415)
Error parsing JSON on line 68: Extra data: line 1 column 419 (char 418)
Error parsing JSON on line 80: Extra data: line 1 column 401 (char 400)
Error parsing JSON on line 82: Extra data: line 1 column 407 (char 406)
Error parsing JSON on line 83: Extra data: line 1 column 398 (char 397)
Error parsing JSON on line 91: Extra data: line 1 column 418 (char 417)
Error parsing JSON on line 92: Extra data: line 1 column 419 (char 418)


AttributeError: 'NoneType' object has no attribute 'head'